## Keras Auto Encoder for Anomaly Detection with Custom Layer for determining Encoder Decoder Reconstruction Cost

### Loading Necessary Files and Libraries

In [1]:
import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Input, Dense, LSTM
from keras.preprocessing.text import Tokenizer
from keras import layers
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras import optimizers
import pickle
import os
from keras.layers import initializers 
from keras.layers import regularizers 
from keras.layers import constraints 
from keras.layers import Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.constraints import max_norm
from keras.layers import LeakyReLU
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.layers import MaxPooling2D, Flatten, Conv2D
from keras.utils import to_categorical
initializer1 = keras.initializers.RandomNormal
initializer2 = keras.initializers.Zeros()
initializer3 = keras.initializers.glorot_uniform(seed=None)
initializer4 = keras.initializers.lecun_normal(seed=None)
%matplotlib inline

Using TensorFlow backend.


## Load Data

In [2]:
data = pd.read_csv("creditcard.csv")

# Try removing both Time and Amount for now
# Amount could become a category cluster or could be normalized
data.drop("Time", inplace=True, axis=1)
data.drop("Amount", inplace=True, axis=1)

print(data.shape)

(284807, 29)


In [3]:
data.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [4]:
data["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
all = data
positive = data[data["Class"] > 0]
negative = data[data["Class"] == 0]

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(284807, 29) (492, 29) (284315, 29) 284807


In [6]:
all.drop("Class", inplace=True, axis=1)
positive.drop("Class", inplace=True, axis=1)
negative.drop("Class", inplace=True, axis=1)

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(284807, 28) (492, 28) (284315, 28) 284807


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Positive is the anomalies - Train AutoEncoder with Negatives

In [7]:
INPUT_SIZE = negative.shape[1]
AUTO_ENCODER_SHAPE = 128

In [8]:
train_size = int(negative.shape[0]*90/100)
negative = negative.sample(frac=1)
X_train = negative.iloc[:train_size]
X_test = negative.iloc[-(negative.shape[0] - train_size):]

# force the model to lower the autoencoder reconstruction cost
Y = np.zeros([X_train.shape[0], 1], dtype = float) # ZEROS MATRIX with shape (,1)

print(X_train.shape, X_test.shape, Y.shape, INPUT_SIZE, X_train.shape[0] + X_test.shape[0])

(255883, 28) (28432, 28) (255883, 1) 28 284315


## Build the Encoder

In [9]:
input = Input(shape=(INPUT_SIZE,))

input_encoder = layers.Dense(AUTO_ENCODER_SHAPE, activation = 'relu')(input)
hidden2 = layers.Dense(64, activation = 'relu')(input_encoder)
encoded = layers.Dense(32, activation = 'relu')(hidden2)

Instructions for updating:
Colocations handled automatically by placer.


## Build the Decoder

In [10]:
hidden3 = Dense(64, activation='relu')(encoded)
output_decoder = Dense(AUTO_ENCODER_SHAPE, activation='relu')(hidden3)
# decoded = Dense(100, activation='relu')(output_decoder)

In [11]:
from keras import backend as K
from keras.layers import Layer

class ReconstructionCostLayer(Layer):

    def __init__(self,output_dim):
        self.output_dim = output_dim
        super(ReconstructionCostLayer, self).__init__()

    def build(self, input_shape):
        super(ReconstructionCostLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        left = K.slice(x, [0, 0], [-1, AUTO_ENCODER_SHAPE])
        right = K.slice(x, [0, AUTO_ENCODER_SHAPE], [-1, -1])
        return K.sum(K.square(left - right), axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

### Concatenate Encoder input and Decoder output

In [12]:
input_encoder_plus_output_decoder = layers.concatenate([input_encoder, output_decoder], axis=1)


### Call the custom layer

In [13]:
reconstruction_cost_layer = ReconstructionCostLayer(1)(input_encoder_plus_output_decoder)

# Build the AutoEncoder with Regression Model

In [14]:
model = Model(input, reconstruction_cost_layer)

#### Visualize Model Architecture

In [15]:
adam = optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-4, decay=0.0, amsgrad=True)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mse'])


In [16]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          3712        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           2080        dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

# Train

In [17]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', 
                                         histogram_freq=0, 
                                         write_graph=True, 
                                         write_images=True)

history = model.fit(X_train, Y, epochs=200, batch_size=50, 
                    verbose=1,
                    shuffle=True,
                    callbacks=[tbCallBack],
                   )  

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
255883/255883 [==============================] - 6s 24us/step - loss: 24.8354 - mean_squared_error: 24.8354
Epoch 2/200
255883/255883 [==============================] - 6s 24us/step - loss: 1.9554 - mean_squared_error: 1.9554
Epoch 3/200
255883/255883 [==============================] - 6s 24us/step - loss: 0.3017 - mean_squared_error: 0.3017
Epoch 4/200
255883/255883 [==============================] - 6s 24us/step - loss: 0.1115 - mean_squared_error: 0.1115
Epoch 5/200
255883/255883 [==============================] - 6s 24us/step - loss: 0.0669 - mean_squared_error: 0.0669
Epoch 6/200
255883/255883 [==============================] - 6s 23us/step - loss: 0.0505 - mean_squared_error: 0.0505
Epoch 7/200
255883/255883 [==============================] - 6s 22us/step - loss: 0.0405 - mean_squared_error: 0.0405
Epoch 8/200
255883/255883 [==============================] - 6s 23us/step - loss: 0.0333 - mean_squared_error: 0.0333
Epoch 

KeyboardInterrupt: 

# Get Negative (Trained) Reconstruction Costs

In [18]:
THRESHOLD = 1

def report_cost(result):
    tot = len(result)
    greater = len(np.where(result > THRESHOLD)[0])
    print("Count: {}\nGreater Than Threeshold: {} {}%\nMean: {}\nStd: {}\nMin: {}\nMax: {}"\
          .format(tot,
                  greater,
                  (tot - greater) * 100 / tot,
                  result.mean(),
                  result.std(),
                  result.min(),
                  result.max()))

In [19]:
reconstruction_costs = model.predict(X_train)
report_cost(reconstruction_costs)

Count: 255883
Greater Than Threeshold: 27 99.98944830254452%
Mean: 0.025433020666241646
Std: 0.04706026613712311
Min: 0.0
Max: 2.513159990310669


In [20]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Negative (New) Reconstruction Costs

In [21]:
reconstruction_costs = model.predict(X_test)
report_cost(reconstruction_costs)

Count: 28432
Greater Than Threeshold: 2 99.99296567248172%
Mean: 0.025260085240006447
Std: 0.045378364622592926
Min: 0.0
Max: 1.1758919954299927


In [22]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Positive Reconstruction Costs

In [23]:
reconstruction_costs = model.predict(positive)
report_cost(reconstruction_costs)

Count: 492
Greater Than Threeshold: 68 86.17886178861788%
Mean: 0.5795853734016418
Std: 1.5153045654296875
Min: 0.0
Max: 13.82332992553711


In [ ]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))